In [ ]:
import pandas as pd
import glob
# import osl
import warnings
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px

In [ ]:
# Use glob to find all Excel files starting with 'Content_'
file_paths = glob.glob('data/Content_*.xlsx')

In [ ]:
sheet_name = "ENGAGEMENT"

# Clean and merge ENGAGEMENT sheets
engagement_dfs = []

for i, f in enumerate(file_paths):
    df = pd.read_excel(f, sheet_name=sheet_name)

    # keep header only from first file
    if i > 0:
        df = df.iloc[1:]

    engagement_dfs.append(df)

print(engagement_dfs[0].tail(5))



In [ ]:
merged_engagement_df = pd.concat(engagement_dfs)
date_col = merged_engagement_df.columns[0]
merged_engagement_df[date_col] = pd.to_datetime(merged_engagement_df[date_col], errors="coerce")
merged_cleaned_engagement_df = merged_engagement_df.drop_duplicates(subset=date_col)
merged_cleaned_engagement_df = merged_cleaned_engagement_df.sort_values(by=date_col)

# merged_cleaned_engagement_df['date'] = '2025-11-11'

print(merged_cleaned_engagement_df.tail(5))

# target_date = '2025-11-25'
# total_engagement = merged_cleaned_engagement_df['Engagements'].sum()
# print(total_engagement)
# merged_engagement_df.to_excel('engagement.xlsx', index=False)



In [ ]:
# target_date = "2025-11-25"

# rows_for_date = merged_cleaned_engagement_df[
#     merged_cleaned_engagement_df["Date"] == target_date
# ]

# print(rows_for_date)


In [ ]:
len(merged_engagement_df)

In [ ]:
sheet_name = "FOLLOWERS"
followers_dfs = []

for f in file_paths:
    df = pd.read_excel(
        f,
        sheet_name=sheet_name,
        skiprows=2
    )

    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

    followers_dfs.append(df)

print(followers_dfs[0].tail(5))

In [ ]:
followers_merged = pd.concat(followers_dfs, ignore_index=True)
followers_merged = followers_merged.drop_duplicates(subset=date_col)
followers_merged = followers_merged.sort_values(by=date_col)
print(followers_merged.tail(5))
# followers_merged.to_excel('followers.xlsx', index=False)

In [ ]:
len(followers_merged)

In [ ]:
sheet_name = "DEMOGRAPHICS"

demographics_dfs = []

for i, f in enumerate(file_paths):
    df = pd.read_excel(
        f,
        sheet_name=sheet_name,
    )
    # keep header only from first file
    if i > 0:
        df = df.iloc[1:]

    demographics_dfs.append(df)

print(demographics_dfs[0].head(5))



    

In [ ]:
# Merge all demographics DataFrames first
merged_demographics_df = pd.concat(demographics_dfs, ignore_index=True)

pct_col = "Percentage"
s = merged_demographics_df[pct_col].astype(str).str.strip()

# extract numeric part from:
# "< 1%", ">95%", "23%", "23.5", "< 0.5 %", etc.
s = s.str.extract(r'([0-9]+(?:\.[0-9]+)?)', expand=False)

# convert to decimal percentage
merged_demographics_df[pct_col] = pd.to_numeric(s, errors="raise") / 100

merged_demographics_df = merged_demographics_df.drop_duplicates(subset="Value")

print(merged_demographics_df.head())
print(merged_demographics_df.dtypes)

# merged_demographics_df.to_excel('demographics.xlsx', index=False)


In [ ]:
merged_demographics_df = pd.concat(demographics_dfs, ignore_index=True)

pct_col = "Percentage"
s = merged_demographics_df[pct_col].astype(str).str.strip()

# extract numeric part from:
# "< 1%", ">95%", "23%", "23.5", "< 0.5 %", etc.
s = s.str.extract(r'([0-9]+(?:\.[0-9]+)?)', expand=False)

# convert to decimal percentage
merged_demographics_df[pct_col] = pd.to_numeric(s, errors="raise") / 100

merged_demographics_df = merged_demographics_df.drop_duplicates(subset="Value")

print(merged_demographics_df.head())
print(merged_demographics_df.dtypes)

# merged_demographics_df.to_excel('demographics.xlsx', index=False)


In [ ]:
len(merged_demographics_df)

In [ ]:
df = []
for f in file_paths: 
    tmp = pd.read_excel(f, sheet_name="TOP POSTS")


    # drop first two rows, then take columns I, J, K
    subset = tmp.iloc[2:, [4,5,6]].copy()
    subset.columns = ["post_url", "post_publish_date", "impressions"]
    subset["export_file"] = f
    df.append(subset)
    print(subset.head(5))

In [ ]:
all_posts = pd.concat(df, ignore_index=True)
# Extract the second date (export end) from export_file
# e.g. Content_2025-11-30_2025-12-15_SteevKundukulangara.xlsx -> 2025-12-15
all_posts["export_end"] = pd.to_datetime(
    all_posts["export_file"].str.extract(r"_(\d{4}-\d{2}-\d{2})_(\d{4}-\d{2}-\d{2})")[1]
)


In [ ]:
# Ensure impressions is numeric
all_posts["impressions"] = pd.to_numeric(all_posts["impressions"], errors="coerce")

# Sort by export_end so latest export comes last
all_posts_sorted = all_posts.sort_values("export_end")

# Take the last row (latest export) per Post URL
latest_posts = (
    all_posts_sorted
    .groupby("post_url", as_index=False)
    .tail(1)
)

# Optional: clean final columns
posts_df = latest_posts[["post_url", "post_publish_date", "impressions", "export_end"]].copy()
posts_df = posts_df.rename(columns={"impressions": "impressions_cumulative"})
print(posts_df.head(5))
# posts_df.to_excel("test.xlsx", index=False)

In [ ]:
first_row = tmp

In [ ]:
df = followers_merged.copy()
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date")

# Use the provided totals if present
if "Total followers" in df.columns:
    df["Total followers"] = pd.to_numeric(df["Total followers"], errors="coerce")
else:
    df["New followers"] = pd.to_numeric(df["New followers"], errors="coerce")
    df["Total followers"] = df["New followers"].cumsum()

df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.to_period("M").astype(str)
df["Quarter"] = df["Date"].dt.to_period("Q").astype(str)


In [ ]:
eng = merged_cleaned_engagement_df.copy()
eng.rename(columns={eng.columns[0]: "Date"}, inplace=True)


eng["Date"] = pd.to_datetime(eng["Date"])
eng["Impressions"] = pd.to_numeric(eng["Impressions"], errors="coerce")
eng["Engagements"] = pd.to_numeric(eng["Engagements"], errors="coerce")

eng = eng.sort_values("Date")

eng["Year"] = eng["Date"].dt.year
eng["Month"] = eng["Date"].dt.to_period("M").astype(str)
eng["Quarter"] = eng["Date"].dt.to_period("Q").astype(str)

eng["Cum engagements"] = eng["Engagements"].cumsum()
eng["Cum impressions"] = eng["Impressions"].cumsum()



In [ ]:
years = sorted(df["Year"].unique())


In [ ]:
# ===================== APP =====================

app = dash.Dash(
    __name__,
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    suppress_callback_exceptions=True,
)


app.layout = dbc.Container(
    [
        # HEADER
        dbc.Row(
            dbc.Col(
                html.Div(
                    [
                        html.Div(
                            [
                                html.H2("Audience Growth & Engagement", className="mb-0"),
                                html.P(
                                    "Followers, impressions, and engagements over time",
                                    className="mb-0",
                                ),
                            ],
                            className="header-title",
                        ),
                        html.Div(
                            dbc.Button(
                                "↻ Refresh",
                                id="refresh-btn",
                                n_clicks=0,
                                color="light",
                                className="btn-refresh",
                            ),
                            className="header-actions",
                        ),
                    ],
                    className="app-header",
                ),
                width=12,
            ),
            className="mb-4",
        ),

        # FILTERS
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Label("Year", className="filter-label"),
                        dcc.Dropdown(
                            id="year-dropdown",
                            options=[{"label": int(y), "value": int(y)} for y in years],
                            value=(int(years[-1]) if years else None),
                            clearable=False,
                            className="filter-dropdown",
                        ),
                    ],
                    md=3,
                ),
                dbc.Col(
                    [
                        html.Label("Date range", className="filter-label"),
                        dcc.DatePickerRange(
                            id="date-range-picker",
                            min_date_allowed=df["Date"].min().date() if not df.empty else None,
                            max_date_allowed=df["Date"].max().date() if not df.empty else None,
                            start_date=None,
                            end_date=None,
                            display_format="YYYY-MM-DD",
                            className="filter-dropdown",
                        ),
                    ],
                    md=4,
                ),
                dbc.Col(
                    [
                        html.Label("Period", className="filter-label"),
                        dcc.Dropdown(
                            id="period-dropdown",
                            options=[
                                {"label": "Daily", "value": "D"},
                                {"label": "Monthly", "value": "M"},
                                {"label": "Quarterly", "value": "Q"},
                                {"label": "Half-year", "value": "H"},
                                {"label": "Year", "value": "Y"},
                            ],
                            value="M",
                            clearable=False,
                            className="filter-dropdown",
                        ),
                    ],
                    md=3,
                ),
            ],
            className="mb-3",
        ),

        # FOLLOWERS KPI CARDS
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Total followers", className="card-title-text"),
                                html.H3(id="total-followers-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-primary",
                    ),
                    md=4,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Followers this period", className="card-title-text"),
                                html.H3(id="period-followers-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-secondary",
                    ),
                    md=4,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("% change vs previous", className="card-title-text"),
                                html.H3(id="pct-change-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-accent",
                    ),
                    md=4,
                ),
            ],
            className="mb-4",
        ),

        # FOLLOWERS CHARTS
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("Cumulative followers", className="chart-title"),
                                dcc.Graph(
                                    id="cum-line",
                                    config={"displayModeBar": False},
                                    style={"height": "28vh"},
                                ),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("New followers by period", className="chart-title"),
                                dcc.Graph(
                                    id="bar-new-followers",
                                    config={"displayModeBar": False},
                                    style={"height": "28vh"},
                                ),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
            ],
            className="mb-4",
        ),

        # ENGAGEMENT + IMPRESSIONS KPI CARDS
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Total engagements", className="card-title-text"),
                                html.H3(id="total-engagements-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-primary",
                    ),
                    md=2,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Engagements this period", className="card-title-text"),
                                html.H3(id="period-engagements-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-secondary",
                    ),
                    md=2,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Engagement rate", className="card-title-text"),
                                html.H3(id="eng-rate-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-accent",
                    ),
                    md=2,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Total impressions", className="card-title-text"),
                                html.H3(id="total-impressions-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-primary",
                    ),
                    md=2,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Impressions this period", className="card-title-text"),
                                html.H3(id="period-impressions-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-secondary",
                    ),
                    md=2,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Impressions rate", className="card-title-text"),
                                html.H3(id="imp-rate-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-accent",
                    ),
                    md=2,
                ),
            ],
            className="mb-4",
        ),

        # ENGAGEMENT CHARTS
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("Engagements over time", className="chart-title"),
                                dcc.Graph(
                                    id="eng-line",
                                    config={"displayModeBar": False},
                                    style={"height": "28vh"},
                                ),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("Weekdays Impressions", className="chart-title"),
                                dcc.Graph(
                                    id="eng-bar",
                                    config={"displayModeBar": False},
                                    style={"height": "28vh"},
                                ),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
            ],
            className="mb-4",
        ),

        # IMPRESSIONS CHARTS
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("Cumulative impressions", className="chart-title"),
                                dcc.Graph(
                                    id="imp-cum-line",
                                    config={"displayModeBar": False},
                                    style={"height": "28vh"},
                                ),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("Impressions by period", className="chart-title"),
                                dcc.Graph(
                                    id="imp-bar",
                                    config={"displayModeBar": False},
                                    style={"height": "28vh"},
                                ),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
            ],
            className="mb-4",
        ),



        html.Div(id="refresh-dummy", style={"display": "none"}),
    ],
    fluid=True,
    className="app-container",
)

# ===================== CALLBACKS =====================

# refresh button
app.clientside_callback(
    """
    function(n_clicks) {
        if (n_clicks && n_clicks > 0) {
            window.location.reload();
        }
        return '';
    }
    """,
    Output("refresh-dummy", "children"),
    Input("refresh-btn", "n_clicks"),
)


@app.callback(
    [
        Output("total-followers-card", "children"),
        Output("period-followers-card", "children"),
        Output("pct-change-card", "children"),
        Output("cum-line", "figure"),
        Output("bar-new-followers", "figure"),
        Output("total-engagements-card", "children"),
        Output("period-engagements-card", "children"),
        Output("eng-rate-card", "children"),
        Output("eng-line", "figure"),
        Output("eng-bar", "figure"),
        Output("imp-cum-line", "figure"),
        Output("imp-bar", "figure"),
        Output("total-impressions-card", "children"),
        Output("period-impressions-card", "children"),
        Output("imp-rate-card", "children"),
    ],
    [
        Input("year-dropdown", "value"),
        Input("date-range-picker", "start_date"),
        Input("date-range-picker", "end_date"),
        Input("period-dropdown", "value"),
    ],
)
def update_dashboard(year, start_date, end_date, period):
    # filter main data by year
    dff = df[df["Year"] == year].copy().sort_values("Date")
    dfe = eng[eng["Year"] == year].copy().sort_values("Date")


    # date range filter
    if start_date:
        s = pd.to_datetime(start_date)
        e = pd.to_datetime(end_date) if end_date else s
        dff = dff[(dff["Date"] >= s) & (dff["Date"] <= e)]
        dfe = dfe[(dfe["Date"] >= s) & (dfe["Date"] <= e)]

    freq = period if period else "M"
    empty_fig = px.scatter()

    if dff.empty or dfe.empty:
        dash_text = "–"
        return (
            dash_text,  # total-followers-card
            dash_text,  # period-followers-card
            dash_text,  # pct-change-card
            empty_fig,  # cum-line
            empty_fig,  # bar-new-followers
            dash_text,  # total-engagements-card
            dash_text,  # period-engagements-card
            dash_text,  # eng-rate-card
            empty_fig,  # eng-line
            empty_fig,  # eng-bar
            empty_fig,  # imp-cum-line
            empty_fig,  # imp-bar
            dash_text,  # total-impressions-card
            dash_text,  # period-impressions-card
            dash_text,  # imp-rate-card
        )


    # ===== FOLLOWERS =====
    total_followers = dff["Total followers"].iloc[-1]

    if freq == "D":
        f_agg = dff.rename(columns={"New followers": "new_followers"}).copy()
        f_x = "Date"
        f_new = f_agg["new_followers"].sum()
        if len(dff) > 1 and dff["Total followers"].iloc[0] != 0:
            f_pct = dff["Total followers"].iloc[-1] / dff["Total followers"].iloc[0] - 1
        else:
            f_pct = 0.0
    elif freq == "M":
        f_agg = (
            dff.groupby("Month")
               .agg(
                   new_followers=("New followers", "sum"),
                   total_followers_end=("Total followers", "last"),
               )
               .reset_index()
        )
        f_x = "Month"
        f_new = f_agg["new_followers"].iloc[-1] if not f_agg.empty else 0
        f_pct = f_agg["total_followers_end"].pct_change().iloc[-1] if len(f_agg) > 1 else 0.0
    elif freq == "Q":
        f_agg = (
            dff.groupby("Quarter")
               .agg(
                   new_followers=("New followers", "sum"),
                   total_followers_end=("Total followers", "last"),
               )
               .reset_index()
        )
        f_x = "Quarter"
        f_new = f_agg["new_followers"].iloc[-1] if not f_agg.empty else 0
        f_pct = f_agg["total_followers_end"].pct_change().iloc[-1] if len(f_agg) > 1 else 0.0
    elif freq == "H":
        dff["Half"] = dff["Date"].apply(lambda dt: f"{dt.year}-H1" if dt.month <= 6 else f"{dt.year}-H2")
        f_agg = (
            dff.groupby("Half")
               .agg(
                   new_followers=("New followers", "sum"),
                   total_followers_end=("Total followers", "last"),
               )
               .reset_index()
        )
        f_x = "Half"
        f_new = f_agg["new_followers"].iloc[-1] if not f_agg.empty else 0
        f_pct = f_agg["total_followers_end"].pct_change().iloc[-1] if len(f_agg) > 1 else 0.0
    elif freq == "Y":
        f_agg = (
            dff.groupby("Year")
               .agg(
                   new_followers=("New followers", "sum"),
                   total_followers_end=("Total followers", "last"),
               )
               .reset_index()
        )
        f_x = "Year"
        f_new = f_agg["new_followers"].iloc[-1] if not f_agg.empty else 0
        f_pct = f_agg["total_followers_end"].pct_change().iloc[-1] if len(f_agg) > 1 else 0.0

    cum_fig = px.line(dff, x="Date", y="Total followers", markers=True)
    cum_fig.update_layout(template="plotly_dark", margin=dict(l=10, r=10, t=10, b=10))

    foll_bar = px.bar(f_agg, x=f_x, y="new_followers")
    foll_bar.update_layout(template="plotly_dark", margin=dict(l=10, r=10, t=10, b=10))
    foll_bar.update_xaxes(tickangle=-35)

    # ===== ENGAGEMENT =====
    total_eng = dfe["Engagements"].sum()

    if freq == "D":
        e_agg = dfe.copy()
        e_x = "Date"
    elif freq == "M":
        e_agg = (
            dfe.groupby("Month")
               .agg(Impressions=("Impressions", "sum"),
                    Engagements=("Engagements", "sum"))
               .reset_index()
        )
        e_x = "Month"
    elif freq == "Q":
        e_agg = (
            dfe.groupby("Quarter")
               .agg(Impressions=("Impressions", "sum"),
                    Engagements=("Engagements", "sum"))
               .reset_index()
        )
        e_x = "Quarter"
    elif freq == "H":
        dfe["Half"] = dfe["Date"].apply(lambda dt: f"{dt.year}-H1" if dt.month <= 6 else f"{dt.year}-H2")
        e_agg = (
            dfe.groupby("Half")
               .agg(Impressions=("Impressions", "sum"),
                    Engagements=("Engagements", "sum"))
               .reset_index()
        )
        e_x = "Half"
    elif freq == "Y":
        e_agg = (
            dfe.groupby("Year")
               .agg(Impressions=("Impressions", "sum"),
                    Engagements=("Engagements", "sum"))
               .reset_index()
        )
        e_x = "Year"

    period_eng = e_agg["Engagements"].iloc[-1] if not e_agg.empty else 0
    total_impr = e_agg["Impressions"].sum() if "Impressions" in e_agg.columns else dfe["Impressions"].sum()
    eng_rate = (total_eng / total_impr) if total_impr else 0.0

    eng_line = px.line(dfe, x="Date", y="Engagements", markers=True)
    eng_line.update_layout(template="plotly_dark", margin=dict(l=10, r=10, t=10, b=10))

    # weekday impressions
    dfe["Weekday"] = dfe["Date"].dt.day_name()
    dow_impressions = (
        dfe.groupby("Weekday")["Impressions"]
           .sum()
           .reindex(["Monday", "Tuesday", "Wednesday", "Thursday",
                     "Friday", "Saturday", "Sunday"])
           .reset_index()
    )
    dow_bar_fig = px.bar(dow_impressions, x="Weekday", y="Impressions")
    dow_bar_fig.update_layout(template="plotly_dark", margin=dict(l=10, r=10, t=10, b=10))
    dow_bar_fig.update_xaxes(tickangle=-35)

    # ===== IMPRESSIONS =====
    imp_cum_fig = px.line(dfe, x="Date", y="Cum impressions", markers=True)
    imp_cum_fig.update_layout(template="plotly_dark", margin=dict(l=10, r=10, t=10, b=10))

    if freq == "D":
        imp_agg = dfe.copy()
        imp_x = "Date"
    elif freq == "M":
        imp_agg = (
            dfe.groupby("Month")
               .agg(Impressions=("Impressions", "sum"))
               .reset_index()
        )
        imp_x = "Month"
    elif freq == "Q":
        imp_agg = (
            dfe.groupby("Quarter")
               .agg(Impressions=("Impressions", "sum"))
               .reset_index()
        )
        imp_x = "Quarter"
    elif freq == "H":
        dfe["Half"] = dfe["Date"].apply(lambda dt: f"{dt.year}-H1" if dt.month <= 6 else f"{dt.year}-H2")
        imp_agg = (
            dfe.groupby("Half")
               .agg(Impressions=("Impressions", "sum"))
               .reset_index()
        )
        imp_x = "Half"
    elif freq == "Y":
        imp_agg = (
            dfe.groupby("Year")
               .agg(Impressions=("Impressions", "sum"))
               .reset_index()
        )
        imp_x = "Year"

    imp_bar_fig = px.bar(imp_agg, x=imp_x, y="Impressions")
    imp_bar_fig.update_layout(template="plotly_dark", margin=dict(l=10, r=10, t=10, b=10))
    imp_bar_fig.update_xaxes(tickangle=-35)

    total_impressions = total_impr
    period_impressions = (
        e_agg["Impressions"].iloc[-1] if "Impressions" in e_agg.columns and not e_agg.empty else 0
    )
    imp_rate = (total_impressions / total_followers) if total_followers else 0.0
    imp_rate_display = f"{imp_rate:0.1f}x"


    return (
        f"{int(total_followers):,}",
        f"{int(f_new):,}",
        f"{f_pct*100:0.1f}%",
        cum_fig,
        foll_bar,
        f"{int(total_eng):,}",
        f"{int(period_eng):,}",
        f"{eng_rate*100:0.2f}%",
        eng_line,
        dow_bar_fig,
        imp_cum_fig,
        imp_bar_fig,
        f"{int(total_impressions):,}",
        f"{int(period_impressions):,}",
        imp_rate_display,
    )


if __name__ == "__main__":
    app.run(debug=True, port=8051)

In [ ]:
dup_dates_eng = eng[eng.duplicated(subset=["Date"], keep=False)]
print(dup_dates_eng.sort_values("Date"))
